# Homework (HW-03) "DE-5.Selenium. Парсинг страниц с эмуляцией поведения человека. Практика".

### Варламов Александр (поток DSPR-131)

## Оглавление <a id="item-zero"></a>

- [**1. Problem(Проблема)**](#item-one)
- [**2. Collection(Сбор данных)**](#item-two)
- [**3. Практика 1. Парсинг теста с сайта TPROGER**](#item-four)
- [**4. Практика 2. Парсинг информации с сайта Hacker News**](#item-five)

## 1. Problem(Проблема) <a id="item-one"></a>

### 1.1. Проблема <a id="item-one-one"></a>

**Парсинг (от анг. parsing — разбор)** — это процесс автоматического сбора данных и их структурирования. Парсинг обычно применяют, когда нужно быстро собрать большой объём данных или же когда данные часто меняются и каждый раз копировать их вручную становится трудозатратным. 

### 1.2. Задачи <a id="item-one-two"></a>

1. Познакомитесь с веб-драйвером Selenium, который выполняет всю ручную работу с браузером автоматизированно, имитируя действия пользователя.

### 1.3. Цели <a id="item-one-three"></a>

1. Распарсить с помощью Selenium тест с результатами и ответами с сайта TPROGER. 
2. Написать скрипт, который будет собирать информацию (рейтинг, время загрузки ссылки, id и ссылку) с первых 10 страниц сайта Hacker News и сохранять её в CSV. 

[Оглавление ⬆️](#item-zero)

## 2. Collection(Сбор данных) <a id="item-two"></a>

### 2.1. Импорт необходимых библиотек, методов и функций <a id="item-two-one"></a>

In [1]:
# Модуль `time` предоставляет функции для работы со временем, например, `time.sleep()` для приостановки выполнения программы.
import time
# Библиотека `requests` позволяет отправлять HTTP-запросы в Python без необходимости вручную добавлять запросы к URL.
import requests
# `BeautifulSoup` из библиотеки `bs4` используется для парсинга HTML и XML документов. Она создает дерево объектов синтаксического разбора для страниц, что упрощает извлечение данных.
from bs4 import BeautifulSoup
# `numpy` - это библиотека для научных вычислений в Python. Она предоставляет многомерные массивы и широкий набор функций для работы с этими массивами.
import numpy as np
# `pandas` - это библиотека для обработки и анализа данных.
import pandas as pd
# Модуль предоставляет функциональность для чтения и записи данных в формате CSV (Comma Separated Values).
import csv
# Модуль pathlib предоставляет классы для обработки файловой системы с использованием объектно-ориентированного подхода.
from pathlib import Path
# Модуль datetime предоставляет классы для управления датами и временем, позволяя выполнять операции, такие как арифметика дат и времени, форматирование и парсинг.
import datetime

# `selenium.webdriver` предоставляет интерфейсы для управления браузерами, включая Chrome, Firefox и другие.
from selenium import webdriver
# `ChromeDriverManager` из пакета `webdriver_manager.chrome` автоматически управляет бинарным файлом ChromeDriver, который необходим для взаимодействия с браузером Chrome через Selenium.
from webdriver_manager.chrome import ChromeDriverManager
# `Service` в контексте `selenium.webdriver.chrome.service` используется для указания пути к исполняемому файлу драйвера браузера.
from selenium.webdriver.chrome.service import Service
# `By` предоставляет набор стратегий поиска элементов в Selenium, например, по ID, имени класса, тегу, CSS-селектору и т.д.
from selenium.webdriver.common.by import By
# `ActionChains` позволяет выполнить серию действий с использованием мыши или клавиатуры.
from selenium.webdriver.common.action_chains import ActionChains
# `NoAlertPresentException` - это исключение, которое выдается, когда ожидается наличие alert, но он не появляется.
from selenium.common.exceptions import NoAlertPresentException
# `expected_conditions` (сокращенно `EC`) содержит набор предусловий для использования с `WebDriverWait`.
from selenium.webdriver.support import expected_conditions as EC
# `WebDriverWait` используется для явного ожидания, когда нужно подождать, пока не выполнится определенное условие (или истечет таймаут).
from selenium.webdriver.support.ui import WebDriverWait
# `NoSuchElementException` - это исключение, выдаваемое Selenium, когда элемент не может быть найден.
from selenium.common.exceptions import NoSuchElementException
# `TimeoutException` - это исключение, выдаваемое Selenium, когда операция не выполняется в заданный таймаут.
from selenium.common.exceptions import TimeoutException

# `fake_useragent` позволяет генерировать случайные, реалистичные заголовки User-Agent для имитации различных устройств при веб-скрейпинге.
from fake_useragent import UserAgent

In [2]:
# фиксируем версии установленных пакетов
!pip freeze > requrements.txt

[Оглавление ⬆️](#item-zero)

### 2.2. Первичный анализ <a id="item-two-two"></a>

In [3]:
# Устанавливаем URL
url = 'https://tproger.ru/quiz/real-programmer'
# Формируем запрос
responce = requests.get(url=url)
responce.encoding = 'utf-8'
# Выводим ответ, если 200 - все ОК
print(responce)


<Response [200]>


In [4]:
# Выводим содержимое страницы
print(responce.text)

<!DOCTYPE html><html lang="ru"><head><meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<title>Тест: а вы точно программист?</title>
<link rel="preconnect" href="https://proxy.tproger.ru">
<link rel="preconnect" href="https://media.tproger.ru">
<style> @font-face { font-family: "Inter override"; src: local("BlinkMacSystemFont"),local("Segoe UI"),local("Roboto"),local("Helvetica Neue"),local("Arial"),local("Noto Sans"); ascent-override: 96.875%; descent-override: 24.14772727%; line-gap-override: 0%; } @font-face { font-family: "Inter override"; src: local("BlinkMacSystemFont"),local("Segoe UI"),local("Roboto"),local("Helvetica Neue"),local("Arial"),local("Noto Sans"); ascent-override: 96.875%; descent-override: 24.14772727%; line-gap-override: 0%; } @font-face { font-family: "Inter override"; src: local("BlinkMacSystemFont"),local("Segoe UI"),local("Roboto"),local("Helvetica Neue"),local("Arial"),local("Noto Sans"); ascent-override: 96.875%; desc

In [5]:
# Парсим содержимое страницы
soup = BeautifulSoup(responce.text, 'html.parser')
print(soup)

<!DOCTYPE html>
<html lang="ru"><head><meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>Тест: а вы точно программист?</title>
<link href="https://proxy.tproger.ru" rel="preconnect"/>
<link href="https://media.tproger.ru" rel="preconnect"/>
<style> @font-face { font-family: "Inter override"; src: local("BlinkMacSystemFont"),local("Segoe UI"),local("Roboto"),local("Helvetica Neue"),local("Arial"),local("Noto Sans"); ascent-override: 96.875%; descent-override: 24.14772727%; line-gap-override: 0%; } @font-face { font-family: "Inter override"; src: local("BlinkMacSystemFont"),local("Segoe UI"),local("Roboto"),local("Helvetica Neue"),local("Arial"),local("Noto Sans"); ascent-override: 96.875%; descent-override: 24.14772727%; line-gap-override: 0%; } @font-face { font-family: "Inter override"; src: local("BlinkMacSystemFont"),local("Segoe UI"),local("Roboto"),local("Helvetica Neue"),local("Arial"),local("Noto Sans"); ascent-override: 96.875%;

[Оглавление ⬆️](#item-zero)

## 3. "Практика 1. Парсинг теста с сайта TPROGER". <a id="item-four"></a>

In [6]:
# Функция для настроек браузера
def set_chrome_options():
    """Настройки параметров и аргументов для браузера Chrome"""
    # Создаем случайный user-agent с помощбю библиотеки fake_useragent
    user_agent = UserAgent().random
    # Создаем объект ChromeOptions
    chrome_options = webdriver.ChromeOptions()
    # Добавляем аргументы и расширения в ChromeOptions
    chrome_options.add_argument('--incognito')
    chrome_options.add_argument('--window-size=1920, 1080')
    chrome_options.add_argument(f'-user-agent={user_agent}')
    
    return chrome_options

url = 'https://tproger.ru/quiz/real-programmer'

with webdriver.Chrome(options=set_chrome_options()) as w:
    w.get(url=url)
    # Устанавливаем задержку, чтобы сайт успел прогрузиться
    time.sleep(5)
    
    # Так как при работе сайта всплывает уведомление о использовании cookies, необходимо его закрыть перед дальнейшей работой 
    # Формируе конструкцию try / except для закрытия всплывающего уведомление о cookies, если оно присутствует
    try:
        cookie_notification_close_button = w.find_element(By.XPATH, '/html/body/div[3]/div[3]/button/div')
        # Устанавливаем задержку
        time.sleep(5)
        cookie_notification_close_button.click()
    except NoSuchElementException:
        pass # Если элемент не найден, значит, уведомление не появилось
    
    # Выводим название теста 
    paragraph = w.find_element(By.CLASS_NAME, 'tp-content-paragraph').text
    print(f'Название теста: {paragraph}')
    print('!' * 200)

    # Фиксируем количество вопросов
    count_answer = np.arange(0, 12)

    # С помощью цикла перебираем вопросы
    for i in count_answer:
        # Выводим наименование вопроса
        question = w.find_element(By.CLASS_NAME, 'tp-quiz-question__text').text
        print(f'Вопрос - {i + 1}: {question}')
        print('-' * 100)
    
        # Выводим варианты ответов на данный вопрос
        answers = w.find_elements(By.CLASS_NAME, 'tp-quiz-answer-list__item')
        for i in range(len(answers)):
            print(f'Вариант ответа {i + 1}: {answers[i].text}')
        # Устанавливаем задержку    
        time.sleep(3)
        
        # Формируем ответ - нажимая на первый вариант ответа
        click_answer = w.find_element(By.XPATH, '//*[@id="quiz-app"]/div/div/div/div[3]/ul/li[1]')
        click_answer.click()
        # Устанавливаем задержку
        time.sleep(3)
        
        # Выводим правильный ответ с % ответа участников 
        true_answer = w.find_element(By.CLASS_NAME, 'tp-answer--result-true').text
        true_answer_cor = true_answer.replace("\n", " ")[:-3] 
        print('-' * 75)
        print(f'Правильный ответ : {true_answer_cor}')
        print()
        # Устанавливаем задержку    
        time.sleep(3)
        
        # Выводим варианты ответов на данный вопрос участников опроса с % ответов
        answers_ = w.find_elements(By.CLASS_NAME, 'tp-quiz-answer-list__item')
        print(f'Как распределились ответы участников:')
        print('-' * 100)
        for i in range(len(answers_)):
            res = answers[i].text.replace("\n", " - ")
            print(f'Вариант ответа {i + 1}: {res}')
        # Устанавливаем задержку    
        time.sleep(3)

        # Нажимаем на кнопку "Далее" для продолжения теста
        click_next = w.find_element(By.CLASS_NAME, 'tp-button--primary')
        w.execute_script('arguments[0].click()', click_next)
        # Устанавливаем задержку    
        time.sleep(3)

        print('*' * 150)
        print()
    
    time.sleep(5)

Название теста: Шуточный тест для проверки, на самом ли деле вы программист или только так считаете. Также отлично вычисляет гуманитариев. Попробуйте.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Вопрос - 1: Начнём с простого. Какую первую программу обычно пишут программисты?
----------------------------------------------------------------------------------------------------
Вариант ответа 1: Hello world
Вариант ответа 2: Сортировку «пузырьком»
Вариант ответа 3: Для взлома аккаунта «ВКонтакте». Такая программа есть у каждого программиста
Вариант ответа 4: Это вопрос с подвохом. Начинают они с починки утюгов, программы тут не при чём.
---------------------------------------------------------------------------
Правильный ответ : Hello world 

Как распределились ответы участников:
-------------------------------------------------------

### <span style="color:blue">Краткие выводы и комментарии:</span>

1. Задание хорошее и полезное, только на курсе сформировано под старый сайт, старые библиотеки. Пожелание - исправить и доработать.

[Оглавление ⬆️](#item-zero)

## 4. "Практика 2. Парсинг информации с сайта Hacker News". <a id="item-five"></a>

### 4.1. Объявим некоторые константы:

In [7]:
# Объявим некоторые константы:
filename = "articles_info.csv" # имя файла, в который будем сохранять результат
#driver_path = "./chromedriver.exe" # укажите ваш путь к chromedriver, который вы загрузили ранее
base_dir = "parser" # "/Users/varlaam/Desktop/Data Science/1. SkillFactory/1. Курс_Profession Data Science/14. Трек ML-инженер/4. Data engineering/5. DE-5_Парсинг данных и сохранение в базу данных заданной предметной области/parser" # укажите директорию, в которую будем сохранять файл
user_agent = "Mozilla/5.0 ..." # ваш user-agent, узнать его можно тут: https://юзерагент.рф, смотреть через браузер Chrome
start_time = time.time() # время начала выполнения программы

### 4.2 Описываем вспомогательных функции.

In [8]:
def get_load_time(article_url, user_agent):
    """
    Функция считает, сколько времени потребуется для загрузки каждой новостной веб-страницы (статьи).
    """
    # будем ждать 3 секунды, иначе выводить exception и присваивать константное значение
    try:
        # меняем значение заголовка. По умолчанию указано, что это python-код
        headers = {
            "User-Agent": user_agent
        }
        # делаем запрос по url статьи article_url
        response = requests.get(
            article_url, headers=headers, stream=True, timeout=3.000
        )
        # получаем время загрузки страницы
        load_time = response.elapsed.total_seconds()
    except Exception as e:
        print(e)
        load_time = ">3"
    return load_time

In [9]:
def write_to_file(output_list, filename, base_dir):
    """
    Функция для сохранения результатов в файл.
    """
    for row in output_list:
        with open(Path(base_dir).joinpath(filename), "a") as csvfile:
            fieldnames = ["id", "load_time", "rank", "points", "title", "url"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(row)

In [10]:
def connect_to_base(browser, page_number):
    """
    Функция ожидает загрузку таблицы на страницу некоторое время и возвращает TRUE в случае её доступности.
    """
    base_url = "https://news.ycombinator.com/news?p={}".format(page_number)
    for connection_attempts in range(1,4): # совершаем 3 попытки подключения
        try:
            browser.get(base_url)
            # ожидаем пока элемент table с id = 'hnmain' будет загружен на страницу
            # затем функция вернет True иначе False 
            WebDriverWait(browser, 5).until(
                EC.presence_of_element_located((By.ID, "hnmain"))
            )
            return True
        except Exception as e:
            print(e)
            print("Error connecting to {}.".format(base_url))
            print("Attempt #{}.".format(connection_attempts))
    return False

In [11]:
def parse_html(html, user_agent):
    """
    Функция парсит страницу с Beautiful Soup, извлекая необходимые атрибуты и записывая их в список.
    """
    
    soup = BeautifulSoup(html, "html.parser")
    output_list = []
   
    # ищем в объекте soup object id, rank, score и title статьи
    tr_blocks = soup.find_all("tr", class_="athing")
    article = 0
    for tr in tr_blocks:
        article_id = tr.get("id") # id
        article_url = tr.find_all("a")[1]["href"]

        # иногда статья располагается не на внешнем сайте, а на ycombinator
        # тогда article_url у нее не полный, а добавочный, с параметрами.
        # например item?id=200933. Для этих случаев будем добавлять url до полного
        if "item?id=" in article_url or "from?site=" in article_url:
            article_url = f"https://news.ycombinator.com/{article_url}"
        load_time = get_load_time(article_url, user_agent)
        # иногда рейтинга может не быть, поэтому воспользуемся try

        try:
            score = soup.find(id=f"score_{article_id}").string
        except Exception as e:
            print(e)
            score = "0 points"
           
        article_info = {
            "id": article_id,
            "load_time": load_time,
            "rank": tr.span.string,
            "points": score,
            "title": tr.find(class_="titleline").text,
            "url": article_url,
        }

        # добавляем информацию о статье в список
        output_list.append(article_info)
        article += 1
    return output_list

In [12]:
# Функция для настроек браузера
def set_chrome_options():
    """Настройки параметров и аргументов для браузера Chrome"""
    # Создаем случайный user-agent с помощбю библиотеки fake_useragent
    user_agent = UserAgent().random
    # Создаем объект ChromeOptions
    chrome_options = webdriver.ChromeOptions()
    # Добавляем аргументы и расширения в ChromeOptions
    chrome_options.add_argument('--incognito')
    chrome_options.add_argument('--window-size=1920, 1080')
    chrome_options.add_argument(f'-user-agent={user_agent}')
    
    return chrome_options

### 4.3 Парсим данные.

In [13]:
# Код из модуля!
# start_time = time.time() # время начала выполнения программы

# # инициализируем веб драйвер
# browser = webdriver.Chrome()

# # перебираем страницы и собираем нужную информацию
# for page_number in range(10):
#     print("getting page " + str(page_number) + "...")
#     if connect_to_base(browser, page_number):
#         time.sleep(5)
#         output_list = parse_html(browser.page_source, user_agent)
#         write_to_file(output_list, filename, base_dir)

#     else:
#         print("Error connecting to hacker news")

# # завершаем работу драйвера
# browser.close()
# time.sleep(1)
# browser.quit()
# end_time = time.time()
# elapsed_time = end_time - start_time
# print("run time: {} seconds".format(elapsed_time))

In [15]:
# Парсим данные с сайта
start_time = time.time() # время начала выполнения программы

with webdriver.Chrome(options=set_chrome_options()) as browser:
    
    # перебираем страницы и собираем нужную информацию
    for page_number in range(10):
        print("getting page " + str(page_number) + "...")
        if connect_to_base(browser, page_number):
            time.sleep(5)
            output_list = parse_html(browser.page_source, user_agent)
            write_to_file(output_list, filename, base_dir)

    else:
        print("Error connecting to hacker news")

end_time = time.time()
elapsed_time = end_time - start_time

print("run time: {} seconds".format(elapsed_time))

getting page 0...
'NoneType' object has no attribute 'string'
HTTPSConnectionPool(host='www.eetimes.com', port=443): Read timed out. (read timeout=3.0)
getting page 1...
'NoneType' object has no attribute 'string'
HTTPSConnectionPool(host='www.eetimes.com', port=443): Read timed out. (read timeout=3.0)
getting page 2...
getting page 3...
('Connection aborted.', OSError(50, 'Network is down'))
getting page 4...
('Connection aborted.', OSError(50, 'Network is down'))
getting page 5...
getting page 6...
('Connection aborted.', OSError(50, 'Network is down'))
getting page 7...
HTTPSConnectionPool(host='prescheme.org', port=443): Max retries exceeded with url: /posts/announcing-the-pre-scheme-restoration.html (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x1685a2010>, 'Connection to prescheme.org timed out. (connect timeout=3.0)'))
HTTPConnectionPool(host='www.neverendingbooks.org', port=80): Max retries exceeded with url: /oulipos-use-of-the-tohoku-paper/ (Ca

In [16]:
# Выводим данные
articles_data = pd.read_csv('parser/articles_info.csv',
    names=["id", "load_time", "rank", "points", "title", "url"],
    encoding='cp1252'
)

display(articles_data)

,id,load_time,rank,points,title,url
0,40756176,0.139382,1.0,184 points,Language is primarily a tool for communication...,https://gwern.net/doc/psychology/linguistics/2...
1,40756034,0.279581,2.0,113 points,Farm: Fast vite compatible build tool written ...,https://www.farmfe.org/
2,40737226,0.318178,3.0,108 points,20x Faster Background Removal in the Browser U...,https://img.ly/blog/browser-background-removal...
3,40755688,0.552224,4.0,55 points,A Forth Story (1995) (groups.google.com),https://groups.google.com/g/comp.lang.forth/c/...
4,40739789,0.501043,5.0,24 points,RuneLite â€“ open-source RuneScape Client (run...,https://runelite.net/
...,...,...,...,...,...,...
295,40711835,0.137275,266.0,149 points,Intel details Skymont (chipsandcheese.com),https://chipsandcheese.com/2024/06/15/intel-de...
296,40720629,0.625457,267.0,308 points,Nature retracts paper that claimed adult stem ...,https://retractionwatch.com/2024/06/18/nature-...
297,40711661,0.287192,268.0,158 points,A Note on Essential Complexity (olano.dev),https://olano.dev/blog/a-note-on-essential-com...
298,40733615,0.932963,269.0,218 points,Nobody knows what's going on (raptitude.com),https://www.raptitude.com/2024/06/nobody-knows...


### 4.4  Задание 1: Добавить к атрибутам количество комментариев по новостям.

In [17]:
# Добавляем в функцию блок для записи в файл дополнительный признак "count_comments"
def write_to_file(output_list, filename, base_dir):
    """
    Функция для сохранения результатов в файл.
    """
    for row in output_list:
        with open(Path(base_dir).joinpath(filename), "a") as csvfile:
            fieldnames = ["id", "load_time", "rank", "points", "title", "url", "count_comments"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(row)

In [18]:
# Добавляем в функцию 'parse_html_comtr' блок по поиску количество комментариев
def parse_html_comtr(html, user_agent):
    """
    Функция парсит страницу с Beautiful Soup, извлекая необходимые атрибуты и записывая их в список.
    """
    
    soup = BeautifulSoup(html, "html.parser")
    output_list = []
   
    # ищем в объекте soup object id, rank, score и title статьи
    tr_blocks = soup.find_all("tr", class_="athing")
    article = 0
    for tr in tr_blocks:
        article_id = tr.get("id") # id
        article_url = tr.find_all("a")[1]["href"]

        # иногда статья располагается не на внешнем сайте, а на ycombinator
        # тогда article_url у нее не полный, а добавочный, с параметрами.
        # например item?id=200933. Для этих случаев будем добавлять url до полного
        if "item?id=" in article_url or "from?site=" in article_url:
            article_url = f"https://news.ycombinator.com/{article_url}"
        load_time = get_load_time(article_url, user_agent)
        # иногда рейтинга может не быть, поэтому воспользуемся try

        try:
            score = soup.find(id=f"score_{article_id}").string
        except Exception as e:
            print(e)
            score = "0 points"
            
        try:
            comments = soup.find_all(href=f"item?id={article_id}")[1].string
            count_comments = comments.replace('Â\xa0', ' ').replace('\xa0', ' ').strip()
        except Exception as e:
            print(e)
            count_comments = '0 comments'
           
        article_info = {
            "id": article_id,
            "load_time": load_time,
            "rank": tr.span.string,
            "points": score,
            "title": tr.find(class_="titleline").text,
            "url": article_url,
            "count_comments": count_comments
        }

        # добавляем информацию о статье в список
        output_list.append(article_info)
        article += 1
    return output_list

# Объявим некоторые константы :
filename_comtr = "articles_info_comtr.csv" # имя файла, в который будем сохранять результат

start_time = time.time() # время начала выполнения программы

with webdriver.Chrome(options=set_chrome_options()) as browser:
    
    # перебираем страницы и собираем нужную информацию
    for page_number in range(10):
        print("getting page " + str(page_number) + "...")
        if connect_to_base(browser, page_number):
            time.sleep(5)
            output_list = parse_html_comtr(browser.page_source, user_agent)
            write_to_file(output_list, filename_comtr, base_dir)

    else:
        print("Error connecting to hacker news")

end_time = time.time()
elapsed_time = end_time - start_time

print("run time: {} seconds".format(elapsed_time))

getting page 0...
'NoneType' object has no attribute 'string'
list index out of range
HTTPSConnectionPool(host='criu.org', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='www.eetimes.com', port=443): Read timed out. (read timeout=3.0)
getting page 1...
'NoneType' object has no attribute 'string'
list index out of range
HTTPSConnectionPool(host='www.eetimes.com', port=443): Read timed out. (read timeout=3.0)
getting page 2...
getting page 3...
('Connection aborted.', OSError(50, 'Network is down'))
getting page 4...
('Connection aborted.', OSError(50, 'Network is down'))
getting page 5...
('Connection aborted.', OSError(50, 'Network is down'))
getting page 6...
('Connection aborted.', OSError(50, 'Network is down'))
getting page 7...
HTTPSConnectionPool(host='prescheme.org', port=443): Max retries exceeded with url: /posts/announcing-the-pre-scheme-restoration.html (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x168479110>, 'Connect

In [19]:
# Выводим данные
articles_data_comtr = pd.read_csv('parser/articles_info_comtr.csv',
    names=["id", "load_time", "rank", "points", "title", "url", "count_comments"],
    encoding='cp1252'
)

display(articles_data_comtr)

,id,load_time,rank,points,title,url,count_comments
0,40756176,0.148606,1.0,187 points,Language is primarily a tool for communication...,https://gwern.net/doc/psychology/linguistics/2...,125 comments
1,40756034,0.343502,2.0,114 points,Farm: Fast vite compatible build tool written ...,https://www.farmfe.org/,51 comments
2,40737226,0.717691,3.0,109 points,20x Faster Background Removal in the Browser U...,https://img.ly/blog/browser-background-removal...,19 comments
3,40755688,0.701175,4.0,55 points,A Forth Story (1995) (groups.google.com),https://groups.google.com/g/comp.lang.forth/c/...,4 comments
4,40739789,0.559641,5.0,25 points,RuneLite â€“ open-source RuneScape Client (run...,https://runelite.net/,16 comments
...,...,...,...,...,...,...,...
295,40711835,0.263821,266.0,149 points,Intel details Skymont (chipsandcheese.com),https://chipsandcheese.com/2024/06/15/intel-de...,59 comments
296,40711661,0.250922,267.0,158 points,A Note on Essential Complexity (olano.dev),https://olano.dev/blog/a-note-on-essential-com...,48 comments
297,40720629,1.127007,268.0,308 points,Nature retracts paper that claimed adult stem ...,https://retractionwatch.com/2024/06/18/nature-...,176 comments
298,40733615,0.634044,269.0,218 points,Nobody knows what's going on (raptitude.com),https://www.raptitude.com/2024/06/nobody-knows...,221 comments


In [20]:
# Выводим информацию
display(articles_data_comtr.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              300 non-null    int64  
 1   load_time       300 non-null    object 
 2   rank            300 non-null    float64
 3   points          300 non-null    object 
 4   title           300 non-null    object 
 5   url             300 non-null    object 
 6   count_comments  300 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 16.5+ KB


None

### 4.4 Многопоточность.

Запустим многопоточность, немного изменив наш основной код — вынесем его в отдельную функцию и будем запускать её в потоке:

In [21]:
# Импортируем необходимые библиотеки
from concurrent.futures import ThreadPoolExecutor, wait # ThreadPoolExecutor - это класс, который предоставляет интерфейс для асинхронного выполнения вызовов функций с помощью пула потоков (thread pool). 
                                                        # Объекты этого класса управляют потоками выполнения, организуя их и контролируя параллельное выполнение.
                                                        # Функция wait используется для ожидания завершения выполнения переданных в параметрах Future объектов. 
                                                        # Future объекты обычно представляют собой асинхронно выполняемые операции и могут быть получены при помощи методов submit или map у экземпляров ThreadPoolExecutor.

# Обернём процедуру парсинга страницы в функцию
def run_process(page_number, filename):
    
    with webdriver.Chrome(options=set_chrome_options()) as browser:
        
        if connect_to_base(browser, page_number):
            time.sleep(5)
            output_list = parse_html_comtr(browser.page_source, user_agent)
            write_to_file(output_list, filename, base_dir)
        
            browser.quit()
        else:
            print("Error connecting to hacker news")
            browser.quit()
       
# Переназначим глобальные переменные        
filename = "articles_info_new.csv" # имя файла, в который будем сохранять результат
#driver_path = "./chromedriver.exe" # укажите ваш путь к chromedriver, который вы загрузили ранее
#base_dir= "parser" # укажите директорию, в которую будем сохранять файл
#user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"

# Фиксируем время старта выполнения кода
start_time = time.time()

futures = []

# Запустим процесс парсинга на нескольких потоках одновременно
with ThreadPoolExecutor() as executor:
    for number in range(10):
        futures.append(
            executor.submit(run_process, number, filename)
        )
       
wait(futures)
# Фиксируем время окончания выполнения кода
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed run time: {} seconds".format(elapsed_time))

('Connection aborted.', OSError(50, 'Network is down'))
Message: 
Stacktrace:
0   chromedriver                        0x000000010299b6b8 chromedriver + 6149816
1   chromedriver                        0x0000000102992cda chromedriver + 6114522
2   chromedriver                        0x000000010241fd5c chromedriver + 400732
3   chromedriver                        0x000000010246caa5 chromedriver + 715429
4   chromedriver                        0x000000010246cbf1 chromedriver + 715761
5   chromedriver                        0x00000001024b1dd4 chromedriver + 998868
6   chromedriver                        0x000000010248fcdd chromedriver + 859357
7   chromedriver                        0x00000001024af0db chromedriver + 987355
8   chromedriver                        0x000000010248fa53 chromedriver + 858707
9   chromedriver                        0x000000010245f6d5 chromedriver + 661205
10  chromedriver                        0x000000010245ff6e chromedriver + 663406
11  chromedriver             

In [22]:
# Выводим данные
articles_data_new = pd.read_csv('parser/articles_info_new.csv',
    names=["id", "load_time", "rank", "points", "title", "url", "count_comments"],
    encoding='cp1252'
)

display(articles_data_new)

,id,load_time,rank,points,title,url,count_comments
0,40733705,0.515763,61.0,234 points,OSRD: Open-Source Railway Designer (osrd.fr),https://osrd.fr/en/,47 comments
1,40742765,>3,62.0,78 points,ShotSpotter does not reduce gun violence or in...,https://www.vitalcitynyc.org/articles/learning...,40 comments
2,40750391,0.464269,63.0,227 points,OpenAI Acquires Rockset (openai.com),https://openai.com/index/openai-acquires-rockset/,74 comments
3,40744098,0.492542,64.0,131 points,Generating audio for video (deepmind.google),https://deepmind.google/discover/blog/generati...,44 comments
4,40741711,0.483281,65.0,11 points,Sloth search for Ruby Weekly â€“ a 100 minute ...,https://slothfinder.apishop.io/,4 comments
...,...,...,...,...,...,...,...
295,40750171,0.680927,26.0,81 points,Twitter and the spread of academic knowledge (...,https://mattsclancy.substack.com/p/twitter-and...,5 comments
296,40719278,0.566982,27.0,32 points,An ingestible device to record gastric electro...,https://medicalxpress.com/news/2024-06-ingesti...,3 comments
297,40739549,0.383092,28.0,37 points,Hobbyists who collect barf bags (cnn.com),https://www.cnn.com/2024/06/19/travel/barf-bag...,27 comments
298,40755444,0.562384,29.0,424 points,My Windows Computer Just Doesn't Feel Like Min...,https://www.howtogeek.com/my-windows-computer-...,387 comments


### <span style="color:blue">Краткие выводы и комментарии:</span>

1. Хорошее задание. Только теории, которая представлена в модуле не достаточно для понимания библиотеки Selenium и представленного кода. Пришлось купить курс "Парсинг.... " на Stepik, частично его пройти (requests, BeautifulSoup, Selenium) и только тогда немного сложился пазл.  

[Оглавление ⬆️](#item-zero)